# 1.回调功能使用技巧.py

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
        model="gpt-4o"
        temperature=0.1,
        max_tokens=50
    )

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
@Time    : 2024/6/01 19:53
@Author  : thezehui@gmail.com
@File    : 1.回调功能使用技巧.py
"""
import time
from typing import Dict, Any, List, Optional
from uuid import UUID

from langchain_core.callbacks import StdOutCallbackHandler, BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


class LLMOpsCallbackHandler(BaseCallbackHandler):
    """自定义LLMOps回调处理器"""
    start_at: float = 0

    def on_chat_model_start(
            self,
            serialized: Dict[str, Any],
            messages: List[List[BaseMessage]],
            *,
            run_id: UUID,
            parent_run_id: Optional[UUID] = None,
            tags: Optional[List[str]] = None,
            metadata: Optional[Dict[str, Any]] = None,
            **kwargs: Any,
    ) -> Any:
        print("聊天模型开始执行了")
        print("serialized:", serialized)
        print("messages:", messages)
        self.start_at = time.time()

    def on_llm_end(
            self,
            response: LLMResult,
            *,
            run_id: UUID,
            parent_run_id: Optional[UUID] = None,
            **kwargs: Any,
    ) -> Any:
        end_at: float = time.time()
        print("完整输出:", response)
        print("程序消耗:", end_at - self.start_at)


# 1.编排prompt
prompt = ChatPromptTemplate.from_template("{query}")

# 3.构建链
chain = {"query": RunnablePassthrough()} | prompt | llm | StrOutputParser()

# 4.调用链并执行
resp = chain.stream(
    "你好，你是？",
    config={"callbacks": [StdOutCallbackHandler(), LLMOpsCallbackHandler()]}
)

for chunk in resp:
    pass



> Entering new RunnableSequence chain...


> Entering new RunnableParallel<query> chain...


> Entering new RunnablePassthrough chain...

> Finished chain.

> Finished chain.


> Entering new ChatPromptTemplate chain...

> Finished chain.
聊天模型开始执行了
serialized: {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'], 'kwargs': {'model_name': 'gpt-4o', 'temperature': 0.1, 'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}, 'max_tokens': 50}, 'name': 'ChatOpenAI'}
messages: [[HumanMessage(content='你好，你是？', additional_kwargs={}, response_metadata={})]]


> Entering new StrOutputParser chain...
完整输出: generations=[[ChatGenerationChunk(text='你好！我是一个人工智能助手，可以帮助回答问题、提供信息或协助完成任务。如果有任何问题或需要帮助的地方，请告诉我！', generation_info={'finish_reason': 'stop', 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_b54fe76834'}, message=AIMessageChunk(content='你好！我是一个人工智能助手，可以帮助回答问题、提供信息或协助完成任务。如果有任何问题或需要帮助的地方，请告诉我！', additional_kwargs={}, response_metadata={'